In [5]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import warnings
from scipy.optimize import minimize
from numpy.linalg import inv, det
import pandas as pd

warnings.filterwarnings('ignore')
font_name = matplotlib.font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
krfont = {'family':font_name, 'size':10}
matplotlib.rc('font', **krfont)
matplotlib.rcParams['axes.unicode_minus'] = False
plt.style.use("ggplot")

In [6]:
class KalmanFilter:
    def __init__(self, A, B, Q, H, G, R):
        """
        상태-공간 모형 매개변수를 초기화함
        
        Parameters
        ----------
        A : array of N x N
        B : array of N
        Q : array of N X N
        H : array of M X N
        R : array of M X M
        (N : 상태(벡터)공간의 차원, M : 측정공간의 차원)
        
        """
        self.A = A
        self.B = B
        self.Q = Q
        self.H = H
        self.R = R
        
        self.m, self.n = H.shape
    
    def predict(self, x, P):
        A, B, Q = self.A, self.B, self.Q
        x_pred = A@x+B
        P_pred = A@P@A.T+Q
        return x_pred, P_pred
    
    def update(self, x_pred, P_pred, z_meas):
        H, R = self.H, self.R
        z_pred = H@x_pred
        v = z_meas-z_pred
        F = H@P_pred@H.T+R
        K = P_pred@H.T@inv(F)
        x_update = x_pred+K@v
        P_update = P_pred-K@H@P_pred
        dF = det(F)
        if dF <= 0:
            dF = 1e-70
            print('Warning : dF <= 0 인 경우가 있습니다.')
        logL = -0.5*np.log(2*np.pi)-0.5*np.log(dF)-0.5*v.T@inv(F)@v
        return x_update, P_update, logL
    
    def setInitState(self, x0, P0):
        """
        상태-공간 모형의 초기 상태 및 공분산 행렬 설정
        
        Parameters
        ----------
        x0 = array of N
        P0 = array of N x N
        (N : 상태공간의 차원)
        
        """
        self.x0 = x0
        self.P0 = P0
    
    def setData(self, data):
        """
        Loglikelihood 계산을 위한 기초데이터 입력
        
        Parameters
        ----------
        data = array of T x M
        (T : 데이터의 길이, M : 측정공간의 차원)
        
        """
        self.data = data
        self.t = data.shape[0]
    
    def calcStates(self):
        """
        상태, 공분산, Log-likelihood 계산
               
        Notes
        -----
        실행 전에 기초데이터/초기상태가 미리 설정되어 있어야 함(setData, setInitState)
        
        """
        x, P, data = self.x0, self.P0, self.data
        self.x_stack, self.P_stack, self.loglik_stack = [], [], []
        for i in range(self.t):
            x_pred, P_pred = self.predict(x, P)
            x, P, loglik = self.update(x_pred, P_pred, data[i])
            self.x_stack.append(x)
            self.P_stack.append(P)
            self.loglik_stack.append(loglik)
    
    def getLoglik(self):
        """
        (minus) Log-likelihood 합 계산
        
        Returns
        -------
        (minus) Log-likehilood : double
               
        Notes
        -----
        실행 전에 상태가 미리 계산되어 있어야 함(calcStates)
        
        """
        self.calcStates()
        return -sum(self.loglik_stack)
    
    def genStates(self, x, period=1):
        A, B, Q = self.A, self.B, self.Q
        x_gen = []
        
        for i in range(period):
            x = A@x+B+np.random.multivariate_normal(mean=np.zeros(self.n), cov=Q)
            x_gen.append(x)
        return x_gen
    
    def genMeasurement(self, x):
        H, G, R = self.H, self.G, self.R
        z = H@x+G+np.random.multivariate_normal(mean=np.zeros(self.m), cov=R)
        return z
    
    def objFunc(self, params):
        kf.setParams(params)
        logL = kf.getLoglik()
        return logL
        
    def estimation(self):
        rst = minimize(fun=self.objFunc, x0=self.params, method='nelder-mead', options={'dist':True, 'maxiter':5000})
        return rst

In [7]:
class DNS(KalmanFilter):
    def __init__(self, params, tau, dt):
        """
        DNS 초기 모수, 입력 금리데이터 만기, 입력 데이터 단위(일, 주, 월 등) 이용해 초기화
        
        Parameters
        ----------
        params : list of double
                 λ, α1, α2, α3, μ1, μ2, μ3, σ11, σ22, σ33 입력
        tau : list of double
              입력 금리데이터의 만기들을 리스트로 하여 입력
        dt : double
             입력 금리데이터 단위(예 : 주=1/52, 일=1/250, 월=1/12 등) 입력
             
        Notes
        -----
        [tau의 길이] = [금리데이터의 1 레코드의 길이] = [측정공간의 차원(M)] 이어야 함
        
        """
        self.tau = tau
        self.dt = dt
        self.setParams(params)
        
    def setParams(self, params):
        """
        DNS 모형 매개변수를 상태-공간 모형 형식에 맞게 설정함
        
        Parameters
        ----------
        params : double
                 λ, ε, κ1, κ2, κ3, θ1, θ2, θ3, σ11, σ22, σ33 입력
        
        """
        self.params = params
        lambda_, eps, kappa1, kappa2, kappa3, theta1, theta2, theta3, sigma11, sigma22, sigma33 = params
        dt, tau = self.dt, self.tau
        
        A = np.array([[1-kappa1*dt, 0, 0],
                      [0, 1-kappa2*dt, 0],
                      [0, 0, 1-kappa3*dt]])
        B = np.array([kappa1*theta1*dt, kappa2*theta2*dt, kappa3*theta3*dt])
        Q = np.array([[sigma11**2, 0, 0],
                      [0, sigma22**2, 0],
                      [0, 0, sigma33**2]])
        H = np.array([[1, (1-np.exp(-lambda_*t))/(lambda_*t), (1-np.exp(-lambda_*t))/(lambda_*t)-np.exp(-lambda_*t)] for t in tau])
        G = np.zeros(len(tau))
        R = np.identity(len(tau))*eps**2
        
        super().__init__(A, B, Q, H, G, R)

In [351]:
# 모수
lambda_, eps, kappa1, kappa2, kappa3, theta1, theta2, theta3, sigma11, sigma22, sigma33 = \
    5e-1,  3e-4,  2e-2,  1e-33, -8e-1, -2e-1, -1e0, -1e-2, 2e-4,  2e-4,  6e-4
# lambda_ += 0.01
dt = 1/250
tau = np.array([0.25, 0.5, 0.75 ,1 , 1.5, 2, 2.5, 3, 4, 5, 7, 10, 20])

# 행렬
A = np.array([[1-kappa1*dt, 0, 0],
                      [0, 1-kappa2*dt, 0],
                      [0, 0, 1-kappa3*dt]])
B = np.array([kappa1*theta1*dt, kappa2*theta2*dt, kappa3*theta3*dt])
Q = np.array([[sigma11**2, 0, 0],
              [0, sigma22**2, 0],
              [0, 0, sigma33**2]])
H = np.array([[1, (1-np.exp(-lambda_*t))/(lambda_*t), (1-np.exp(-lambda_*t))/(lambda_*t)-np.exp(-lambda_*t)] for t in tau])
G = np.zeros(len(tau))
R = np.identity(len(tau))*eps**2

In [303]:
# Inputs
# dx_prev1, dx_prev2, dx_prev3 = 0, 0, 0
# dP_prev11, dP_prev12, dP_prev13 = 0, 0, 0
# dP_prev31, dP_prev22, dP_prev23 = 0, 0, 0
# dP_prev31, dP_prev32, dP_prev33 = 0, 0, 0

x_prev = np.array([1,1,1])
P_prev = np.identity(3)
dx_prev = np.zeros(3)
dP_prev = np.zeros(shape=(3,3))
z_meas = np.array([0.0253, 0.0267, 0.0278, 0.0288, 0.0321, 0.0339, 0.0346, 0.0349, 0.04, 0.0417, 0.044, 0.046, 0.0473])

# predict
x_pred = A@x_prev+B
P_pred = A@P_prev@A.T+Q

# update
z_pred = H@x_pred
F = H@P_pred@H.T+R
v = z_meas-z_pred
F_inv = inv(F)
K = P_pred@H.T@F_inv

In [257]:
# dx_pred/dx_prev
dx_pred_dx_prev1 = A@np.array([1,0,0])
dx_pred_dx_prev2 = A@np.array([0,1,0])
dx_pred_dx_prev3 = A@np.array([0,0,1])

# dz_pred/dx_prev
dz_pred_dx_prev1 = H@dx_pred_dx_prev1
dz_pred_dx_prev2 = H@dx_pred_dx_prev2
dz_pred_dx_prev3 = H@dx_pred_dx_prev3

# dv/dx_prev
dv_dx_prev1 = -dz_pred_dx_prev1
dv_dx_prev2 = -dz_pred_dx_prev2
dv_dx_prev3 = -dz_pred_dx_prev3

# dlogL/dx_prev
dlogL_dx_prev1 = -0.5*(dv_dx_prev1.T@F_inv@v+v.T@F_inv@dv_dx_prev1)
dlogL_dx_prev2 = -0.5*(dv_dx_prev2.T@F_inv@v+v.T@F_inv@dv_dx_prev2)
dlogL_dx_prev3 = -0.5*(dv_dx_prev3.T@F_inv@v+v.T@F_inv@dv_dx_prev3)

# dx_update/dx_prev
dx_update_dx_prev1 = dx_pred_dx_prev1+K@dv_pred_dx_prev1
dx_update_dx_prev2 = dx_pred_dx_prev2+K@dv_pred_dx_prev2
dx_update_dx_prev3 = dx_pred_dx_prev3+K@dv_pred_dx_prev3

In [189]:
# dP_pred/dP_prev
dP_pred_dP_prev11 = A@np.array([[1,0,0],[0,0,0],[0,0,0]])
dP_pred_dP_prev12 = A@np.array([[0,1,0],[0,0,0],[0,0,0]])
dP_pred_dP_prev13 = A@np.array([[0,0,1],[0,0,0],[0,0,0]])
dP_pred_dP_prev21 = A@np.array([[0,0,0],[1,0,0],[0,0,0]])
dP_pred_dP_prev22 = A@np.array([[0,0,0],[0,1,0],[0,0,0]])
dP_pred_dP_prev23 = A@np.array([[0,0,0],[0,0,1],[0,0,0]])
dP_pred_dP_prev31 = A@np.array([[0,0,0],[0,0,0],[1,0,0]])
dP_pred_dP_prev32 = A@np.array([[0,0,0],[0,0,0],[0,1,0]])
dP_pred_dP_prev33 = A@np.array([[0,0,0],[0,0,0],[0,0,1]])

# dF/dP_prev
dF_pred_dP_prev11 = H@dP_pred_dP_prev11@H.T
dF_pred_dP_prev12 = H@dP_pred_dP_prev12@H.T
dF_pred_dP_prev13 = H@dP_pred_dP_prev13@H.T
dF_pred_dP_prev21 = H@dP_pred_dP_prev21@H.T
dF_pred_dP_prev22 = H@dP_pred_dP_prev22@H.T
dF_pred_dP_prev23 = H@dP_pred_dP_prev23@H.T
dF_pred_dP_prev31 = H@dP_pred_dP_prev31@H.T
dF_pred_dP_prev32 = H@dP_pred_dP_prev32@H.T
dF_pred_dP_prev33 = H@dP_pred_dP_prev33@H.T

# dlogL/dP_prev
dlogL_dP_prev11 = -0.5*np.trace(F_inv@dF_pred_dP_prev11)-0.5*v.T@(-F_inv@dF_pred_dP_prev11@F_inv)@v
dlogL_dP_prev12 = -0.5*np.trace(F_inv@dF_pred_dP_prev12)-0.5*v.T@(-F_inv@dF_pred_dP_prev12@F_inv)@v
dlogL_dP_prev13 = -0.5*np.trace(F_inv@dF_pred_dP_prev13)-0.5*v.T@(-F_inv@dF_pred_dP_prev13@F_inv)@v
dlogL_dP_prev21 = -0.5*np.trace(F_inv@dF_pred_dP_prev21)-0.5*v.T@(-F_inv@dF_pred_dP_prev21@F_inv)@v
dlogL_dP_prev22 = -0.5*np.trace(F_inv@dF_pred_dP_prev22)-0.5*v.T@(-F_inv@dF_pred_dP_prev22@F_inv)@v
dlogL_dP_prev23 = -0.5*np.trace(F_inv@dF_pred_dP_prev23)-0.5*v.T@(-F_inv@dF_pred_dP_prev23@F_inv)@v
dlogL_dP_prev31 = -0.5*np.trace(F_inv@dF_pred_dP_prev31)-0.5*v.T@(-F_inv@dF_pred_dP_prev31@F_inv)@v
dlogL_dP_prev32 = -0.5*np.trace(F_inv@dF_pred_dP_prev32)-0.5*v.T@(-F_inv@dF_pred_dP_prev32@F_inv)@v
dlogL_dP_prev33 = -0.5*np.trace(F_inv@dF_pred_dP_prev33)-0.5*v.T@(-F_inv@dF_pred_dP_prev33@F_inv)@v

# dK/dP_prev
dK_pred_dP_prev11 = dP_pred_dP_prev11@H.T@F_inv+P_pred@H.T@(-F_inv@dF_pred_dP_prev11@F_inv)
dK_pred_dP_prev12 = dP_pred_dP_prev12@H.T@F_inv+P_pred@H.T@(-F_inv@dF_pred_dP_prev12@F_inv)
dK_pred_dP_prev13 = dP_pred_dP_prev13@H.T@F_inv+P_pred@H.T@(-F_inv@dF_pred_dP_prev13@F_inv)
dK_pred_dP_prev21 = dP_pred_dP_prev21@H.T@F_inv+P_pred@H.T@(-F_inv@dF_pred_dP_prev21@F_inv)
dK_pred_dP_prev22 = dP_pred_dP_prev22@H.T@F_inv+P_pred@H.T@(-F_inv@dF_pred_dP_prev22@F_inv)
dK_pred_dP_prev23 = dP_pred_dP_prev23@H.T@F_inv+P_pred@H.T@(-F_inv@dF_pred_dP_prev23@F_inv)
dK_pred_dP_prev31 = dP_pred_dP_prev31@H.T@F_inv+P_pred@H.T@(-F_inv@dF_pred_dP_prev31@F_inv)
dK_pred_dP_prev32 = dP_pred_dP_prev32@H.T@F_inv+P_pred@H.T@(-F_inv@dF_pred_dP_prev32@F_inv)
dK_pred_dP_prev33 = dP_pred_dP_prev33@H.T@F_inv+P_pred@H.T@(-F_inv@dF_pred_dP_prev33@F_inv)

# dP_update/d_prev
dP_update_dP_prev11 = dP_pred_dP_prev11-(dK_pred_dP_prev11@H@P_pred+K@H@dP_pred_dP_prev11)
dP_update_dP_prev12 = dP_pred_dP_prev12-(dK_pred_dP_prev12@H@P_pred+K@H@dP_pred_dP_prev12)
dP_update_dP_prev13 = dP_pred_dP_prev13-(dK_pred_dP_prev13@H@P_pred+K@H@dP_pred_dP_prev13)
dP_update_dP_prev21 = dP_pred_dP_prev21-(dK_pred_dP_prev21@H@P_pred+K@H@dP_pred_dP_prev21)
dP_update_dP_prev22 = dP_pred_dP_prev22-(dK_pred_dP_prev22@H@P_pred+K@H@dP_pred_dP_prev22)
dP_update_dP_prev23 = dP_pred_dP_prev23-(dK_pred_dP_prev23@H@P_pred+K@H@dP_pred_dP_prev23)
dP_update_dP_prev31 = dP_pred_dP_prev31-(dK_pred_dP_prev31@H@P_pred+K@H@dP_pred_dP_prev31)
dP_update_dP_prev32 = dP_pred_dP_prev32-(dK_pred_dP_prev32@H@P_pred+K@H@dP_pred_dP_prev32)
dP_update_dP_prev33 = dP_pred_dP_prev33-(dK_pred_dP_prev33@H@P_pred+K@H@dP_pred_dP_prev33)

In [190]:
# dA/dκ
dA_dkappa1 = np.array([[-dt, 0, 0], [0, 0, 0], [0, 0, 0]])
dA_dkappa2 = np.array([[0, 0, 0], [0, -dt, 0], [0, 0, 0]])
dA_dkappa3 = np.array([[0, 0, 0], [0, 0, 0], [0, 0, -dt]])

# dB/dκ
dB_dkappa1 = np.array([theta1*dt, 0, 0])
dB_dkappa2 = np.array([0, theta2*dt, 0])
dB_dkappa3 = np.array([0, 0, theta3*dt])

# dx_pred/dκ
dx_pred_dkappa1 = dA_dkappa1@x_prev + dB_dkappa1
dx_pred_dkappa2 = dA_dkappa2@x_prev + dB_dkappa2
dx_pred_dkappa3 = dA_dkappa3@x_prev + dB_dkappa3

# dP_pred/dκ
dP_pred_dkappa1 = dA_dkappa1@P_prev@A.T + A@P_prev@dA_dkappa1.T
dP_pred_dkappa2 = dA_dkappa2@P_prev@A.T + A@P_prev@dA_dkappa2.T
dP_pred_dkappa3 = dA_dkappa3@P_prev@A.T + A@P_prev@dA_dkappa3.T

# dz_pred/dκ
dz_pred_dkappa1 = H@dx_pred_dkappa1
dz_pred_dkappa2 = H@dx_pred_dkappa2
dz_pred_dkappa3 = H@dx_pred_dkappa3

# dv/dκ
dv_dkappa1 = -dz_pred_dkappa1
dv_dkappa2 = -dz_pred_dkappa2
dv_dkappa3 = -dz_pred_dkappa3

# dF/dκ
dF_dkappa1 = H@dP_pred_dkappa1@H.T
dF_dkappa2 = H@dP_pred_dkappa2@H.T
dF_dkappa3 = H@dP_pred_dkappa3@H.T

# dlogL/dκ
dlogL_dkappa1 = -0.5*np.trace(F_inv@dF_dkappa1)-0.5*(dv_dkappa1.T@F_inv@v+v.T@(-F_inv@dF_dkappa1@F_inv)@v+v.T@F_inv@dv_dkappa1)
dlogL_dkappa2 = -0.5*np.trace(F_inv@dF_dkappa2)-0.5*(dv_dkappa2.T@F_inv@v+v.T@(-F_inv@dF_dkappa2@F_inv)@v+v.T@F_inv@dv_dkappa2)
dlogL_dkappa3 = -0.5*np.trace(F_inv@dF_dkappa3)-0.5*(dv_dkappa3.T@F_inv@v+v.T@(-F_inv@dF_dkappa3@F_inv)@v+v.T@F_inv@dv_dkappa3)

# dK/dκ
dK_dkappa1 = dP_pred_dkappa1@H.T@F_inv+P_pred@H.T@(-F_inv@dF_dkappa1@F_inv)
dK_dkappa2 = dP_pred_dkappa2@H.T@F_inv+P_pred@H.T@(-F_inv@dF_dkappa2@F_inv)
dK_dkappa3 = dP_pred_dkappa3@H.T@F_inv+P_pred@H.T@(-F_inv@dF_dkappa3@F_inv)

# dx_update/dκ
dx_update_dkappa1 = dx_pred_dkappa1+dK_dkappa1@v+K@dv_dkappa1
dx_update_dkappa2 = dx_pred_dkappa2+dK_dkappa1@v+K@dv_dkappa2
dx_update_dkappa3 = dx_pred_dkappa3+dK_dkappa1@v+K@dv_dkappa3

# dP_update/dκ
dP_update_dkappa1 = dP_pred_dkappa1-(dK_dkappa1@H@P_pred+K@H@dP_pred_dkappa1)
dP_update_dkappa2 = dP_pred_dkappa2-(dK_dkappa2@H@P_pred+K@H@dP_pred_dkappa2)
dP_update_dkappa3 = dP_pred_dkappa3-(dK_dkappa3@H@P_pred+K@H@dP_pred_dkappa3)

In [140]:
# dB/dθ
dB_dtheta1 = np.array([theta1*dt, 0, 0])
dB_dtheta2 = np.array([0, theta2*dt, 0])
dB_dtheta3 = np.array([0, 0, theta3*dt])

# dx_pred/dθ
dx_pred_dtheta1 = dB_dtheta1
dx_pred_dtheta2 = dB_dtheta2
dx_pred_dtheta3 = dB_dtheta3

# dz_pred/dθ
dz_pred_dtheta1 = H@dx_pred_dtheta1
dz_pred_dtheta2 = H@dx_pred_dtheta2
dz_pred_dtheta3 = H@dx_pred_dtheta3

# dv/dθ
dv_dtheta1 = -dz_pred_dtheta1
dv_dtheta2 = -dz_pred_dtheta2
dv_dtheta3 = -dz_pred_dtheta3

# dlogL/dθ
dlogL_dtheta1 = -0.5*(dv_dtheta1.T@F_inv@v+v.T@F_inv@dv_dtheta1)
dlogL_dtheta2 = -0.5*(dv_dtheta2.T@F_inv@v+v.T@F_inv@dv_dtheta2)
dlogL_dtheta3 = -0.5*(dv_dtheta3.T@F_inv@v+v.T@F_inv@dv_dtheta3)

# dx_update/dθ
dx_update_theta1 = dx_pred_dtheta1
dx_update_theta2 = dx_pred_dtheta2
dx_update_theta3 = dx_pred_dtheta3

array([ 0.e+00,  0.e+00, -4.e-05])

In [242]:
# dQ/dσ
dQ_dsigma11 = np.array([[2*sigma11, 0, 0], [0, 0, 0], [0, 0, 0]])
dQ_dsigma22 = np.array([[0, 0, 0], [0, 2*sigma22, 0], [0, 0, 0]])
dQ_dsigma33 = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 2*sigma33]])

# dP_pred/dσ
dP_pred_dsigma11 = dQ_dsigma11
dP_pred_dsigma22 = dQ_dsigma22
dP_pred_dsigma33 = dQ_dsigma33

# dF/dσ
dF_dsigma11 = H@dP_pred_dsigma11@H.T
dF_dsigma22 = H@dP_pred_dsigma22@H.T
dF_dsigma33 = H@dP_pred_dsigma33@H.T

# dlogL/dσ
dlogL_dsigma11 = -0.5*np.trace(F_inv*dF_dsigma11)-0.5*v.T@(-F_inv@dF_dsigma11@F_inv)@v
dlogL_dsigma22 = -0.5*np.trace(F_inv*dF_dsigma22)-0.5*v.T@(-F_inv@dF_dsigma22@F_inv)@v
dlogL_dsigma33 = -0.5*np.trace(F_inv*dF_dsigma33)-0.5*v.T@(-F_inv@dF_dsigma33@F_inv)@v

# dK/dσ
dK_dsigma11 = P_pred@H.T@(-F_inv@dF_dsigma11@F_inv)
dK_dsigma22 = P_pred@H.T@(-F_inv@dF_dsigma22@F_inv)
dK_dsigma33 = P_pred@H.T@(-F_inv@dF_dsigma33@F_inv)

# dP_update/dσ
dP_update_dsigma11 = dP_pred_dsigma11-(dK_dsigma11@H@P_pred+K@H@dP_pred_dsigma11)
dP_update_dsigma22 = dP_pred_dsigma22-(dK_dsigma22@H@P_pred+K@H@dP_pred_dsigma22)
dP_update_dsigma33 = dP_pred_dsigma33-(dK_dsigma33@H@P_pred+K@H@dP_pred_dsigma33)

In [279]:
# dH/dλ
dH_dlambda = np.array([[0, np.exp(-lambda_*t)-(1-np.exp(-lambda_*t))/(t*lambda_**2), 2*np.exp(-lambda_*t)-(1-np.exp(-lambda_*t))/(t*lambda_**2)] for t in tau])

# dz_pred/dλ
dz_pred_dlambda = dH_dlambda@x_pred

# dv/dλ
dv_dlambda = -dz_pred_dlambda

# dF/dλ
dF_dlambda = (dH_dlambda@P_pred@H.T+H@P_pred@dH_dlambda.T)

# dlogL/dλ
dlogL_dlambda = -0.5*np.trace(F_inv@dF_dlambda)-0.5*(dv_dlambda.T@F_inv@v+v.T@(-F_inv@dF_dlambda@F_inv)@v+v.T@F_inv@dv_dlambda)

# dK/dλ
dK_dlambda = P_pred@(dH_dlambda.T@F_inv+H.T@-F_inv@dF_dlambda@F_inv)

# dx_update/dλ
dx_update_dlambda = dK_dlambda@v+K@dv_dlambda

# dP_update/dλ
dP_update_dlambda = -(dK_dlambda@H+K@dH_dlambda)@P_pred

# Total
dx_update_dx_prev = dx_update_dx_prev1*dx_prev1+dx_update_dx_prev2*dx_prev2+dx_update_dx_prev3*dx_prev3
dP_update_dP_prev = dP_update_dP_prev11*dP_prev11+dP_update_dP_prev12*dP_prev12+dP_update_dP_prev13*dP_prev13 \
    + dP_update_dP_prev21*dP_prev21+dP_update_dP_prev22*dP_prev22+dP_update_dP_prev13*dP_prev13 \
    + dP_update_dP_prev31*dP_prev31+dP_update_dP_prev32*dP_prev32+dP_update_dP_prev33*dP_prev33

dlogL_dx_prev = dlogL_dx_prev1*dx_prev1+dlogL_dx_prev2*dx_prev2+dlogL_dx_prev3*dx_prev3

dlogL_dP_prev = dlogL_dP_prev11*dP_prev11+dlogL_dP_prev12*dP_prev12+dlogL_dP_prev13*dP_prev13 \
    + dlogL_dP_prev21*dP_prev21+dlogL_dP_prev22*dP_prev22+dlogL_dP_prev23*dP_prev23 \
    + dlogL_dP_prev31*dP_prev31+dlogL_dP_prev32*dP_prev32+dlogL_dP_prev33*dP_prev33

dlogL = dlogL_dlambda + dlogL_dx_prev + dlogL_dP_prev

dx_prev1, dx_prev2, dx_prev3 = dx_update_dlambda+dx_update_dx_prev
dP_prev11, dP_prev12, dP_prev13 = dP_update_dlambda[0]+dP_update_dP_prev[0]
dP_prev21, dP_prev22, dP_prev23 = dP_update_dlambda[1]+dP_update_dP_prev[1]
dP_prev31, dP_prev32, dP_prev33 = dP_update_dlambda[2]+dP_update_dP_prev[2]

In [343]:
x_prev = np.array([1,1,1])
P_prev = np.identity(3)

logL_sum = 0
dlogL_sum = 0
for i in range(len(data)):
    x_pred, P_pred = predict(x_prev, P_prev)
    z_meas = data[i]
    x_update, P_update, logL = update(x_pred, P_pred, z_meas)
    
    ####################
    dx_prev1, dx_prev2, dx_prev3 = dx_prev
    dP_prev11, dP_prev12, dP_prev13 = dP_prev[0]
    dP_prev21, dP_prev22, dP_prev23 = dP_prev[1]
    dP_prev31, dP_prev32, dP_prev33 = dP_prev[2]

    # dH/dλ
    dH_dlambda = np.array([[0, np.exp(-lambda_*t)-(1-np.exp(-lambda_*t))/(t*lambda_**2), 2*np.exp(-lambda_*t)-(1-np.exp(-lambda_*t))/(t*lambda_**2)] for t in tau])

    # dz_pred/dλ
    dz_pred_dlambda = dH_dlambda@x_pred

    # dv/dλ
    dv_dlambda = -dz_pred_dlambda

    # dF/dλ
    dF_dlambda = (dH_dlambda@P_pred@H.T+H@P_pred@dH_dlambda.T)

    # dlogL/dλ
    dlogL_dlambda = -0.5*np.trace(F_inv@dF_dlambda)-0.5*(dv_dlambda.T@F_inv@v+v.T@(-F_inv@dF_dlambda@F_inv)@v+v.T@F_inv@dv_dlambda)

    # dK/dλ
    dK_dlambda = P_pred@(dH_dlambda.T@F_inv+H.T@-F_inv@dF_dlambda@F_inv)

    # dx_update/dλ
    dx_update_dlambda = dK_dlambda@v+K@dv_dlambda

    # dP_update/dλ
    dP_update_dlambda = -(dK_dlambda@H+K@dH_dlambda)@P_pred

    # Total
    dx_update_dx_prev = dx_update_dx_prev1*dx_prev1+dx_update_dx_prev2*dx_prev2+dx_update_dx_prev3*dx_prev3
    dP_update_dP_prev = dP_update_dP_prev11*dP_prev11+dP_update_dP_prev12*dP_prev12+dP_update_dP_prev13*dP_prev13 \
        + dP_update_dP_prev21*dP_prev21+dP_update_dP_prev22*dP_prev22+dP_update_dP_prev13*dP_prev13 \
        + dP_update_dP_prev31*dP_prev31+dP_update_dP_prev32*dP_prev32+dP_update_dP_prev33*dP_prev33

    dlogL_dx_prev = dlogL_dx_prev1*dx_prev1+dlogL_dx_prev2*dx_prev2+dlogL_dx_prev3*dx_prev3

    dlogL_dP_prev = dlogL_dP_prev11*dP_prev11+dlogL_dP_prev12*dP_prev12+dlogL_dP_prev13*dP_prev13 \
        + dlogL_dP_prev21*dP_prev21+dlogL_dP_prev22*dP_prev22+dlogL_dP_prev23*dP_prev23 \
        + dlogL_dP_prev31*dP_prev31+dlogL_dP_prev32*dP_prev32+dlogL_dP_prev33*dP_prev33

    dlogL = dlogL_dlambda + dlogL_dx_prev + dlogL_dP_prev

    dx_update = dx_update_dlambda+dx_update_dx_prev
    dP_update = dP_update_dlambda+dP_update_dP_prev
    ####################
    
    logL_sum += logL
    x_prev, P_prev = x_update, P_update
    
    dlogL_sum += dlogL  
    dx_prev, dP_prev = dx_update, dP_update

In [342]:
def predict(x, P):
    x_pred = A@x+B
    P_pred = A@P@A.T+Q
    return x_pred, P_pred

def update(x_pred, P_pred, z_meas):
    z_pred = H@x_pred
    v = z_meas-z_pred
    F = H@P_pred@H.T+R
    F_inv = inv(F)
    K = P_pred@H.T@F_inv
    x_update = x_pred+K@v
    P_update = P_pred-K@H@P_pred
    dF = det(F)
    if dF <= 0:
        dF = 1e-70
        print('Warning : dF <= 0 인 경우가 있습니다.')
    logL = -0.5*np.log(2*np.pi)-0.5*np.log(dF)-0.5*v.T@inv(F)@v
    return x_update, P_update, logL

In [224]:
# dR/dε
dR_deps = np.identity(len(tau))*(2*eps)

# dF/dε
dF_deps = dR_deps

# dlogL/dε
dlogL_deps = -0.5*np.trace(F_inv@dF_deps)-0.5*v.T@(-F_inv@dF_deps@F_inv)@v

# dK/dε
dK_deps = P_pred@H.T@(-F_inv@dF_deps@F_inv)

# dx_update/dε
dx_update_deps = dK_deps@v

# dP_update/dε
dP_update_deps = -dK_deps@H@P_pred